In [1]:
import pandas as pd
import numpy as np
import random
from math import sqrt
dataFrame = pd.read_csv("subset_16P.csv")
dataFrame["Personality"]=dataFrame["Personality"].map({"ESTJ":0,"ENTJ":1,"ESFJ":2,"ENFJ":3,"ISTJ":4,"ISFJ":5,"INTJ":6,"INFJ":7,"ESTP":8,"ESFP":9,"ENTP":10,"ENFP":11,
         "ISTP":12,"ISFP":13,"INTP":14,"INFP":15}).astype(int)

dataFrame2 = pd.read_csv("energy_efficiency_data.csv")
dataFrame

,Response Id,You regularly make new friends.,You spend a lot of your free time exploring various random topics that pique your interest,Seeing other people cry can easily make you feel like you want to cry too,You often make a backup plan for a backup plan.,"You usually stay calm, even under a lot of pressure","At social events, you rarely try to introduce yourself to new people and mostly talk to the ones you already know",You prefer to completely finish one project before starting another.,You are very sentimental.,You like to use organizing tools like schedules and lists.,...,You believe that pondering abstract philosophical questions is a waste of time.,"You feel more drawn to places with busy, bustling atmospheres than quiet, intimate places.",You know at first glance how someone is feeling.,You often feel overwhelmed.,You complete things methodically without skipping over any steps.,You are very intrigued by things labeled as controversial.,You would pass along a good opportunity if you thought someone else needed it more.,You struggle with deadlines.,You feel confident that things will work out for you.,Personality
0,35874,-1,0,-1,1,-1,-2,-2,0,-1,...,0,3,0,0,0,0,1,-1,0,10
1,42624,0,0,1,0,0,0,-1,0,0,...,0,2,0,0,0,0,-1,-3,2,14
2,55199,0,0,-2,-1,2,-2,0,0,-1,...,0,0,0,1,0,0,3,0,0,8
3,52983,0,0,0,1,-2,-1,0,0,1,...,1,1,0,-1,0,-1,2,-2,0,10
4,22864,0,0,2,1,0,-2,-1,0,1,...,1,-2,0,1,0,0,0,-2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1898,0,0,1,0,-2,0,0,0,3,...,0,2,0,1,0,0,0,0,2,7
9996,36103,0,-1,-2,-2,-2,1,0,0,-2,...,0,-1,0,-3,0,0,2,0,-3,0
9997,46766,0,0,2,1,2,-1,0,0,-1,...,-1,-1,0,0,0,0,2,0,1,11
9998,2050,0,0,1,1,0,1,-1,0,1,...,0,-2,0,3,0,0,2,0,1,7


In [2]:
dataFrame2

,Relative_Compactness,Surface_Area,Wall_Area,Roof_Area,Overall_Height,Orientation,Glazing_Area,Glazing_Area_Distribution,Heating_Load,Cooling_Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28
...,...,...,...,...,...,...,...,...,...,...
763,0.64,784.0,343.0,220.50,3.5,5,0.4,5,17.88,21.40
764,0.62,808.5,367.5,220.50,3.5,2,0.4,5,16.54,16.88
765,0.62,808.5,367.5,220.50,3.5,3,0.4,5,16.44,17.11
766,0.62,808.5,367.5,220.50,3.5,4,0.4,5,16.48,16.61


In [3]:
class Classification:
    def __init__(self, kNN, fold, data,accuracyTab,precisionTab,recallTab, weighted=False,featureScale = False):
        self.kNN = kNN
        self.fold = fold
        self.weighted = weighted
        self.data = data
        self.featureScale = featureScale
        self.accuracyTab= accuracyTab
        self.precisionTab= precisionTab
        self.recallTab= recallTab

        
    def featureScaling(self,dataSet):#feature normalization function
        maxVal = np.max(dataSet)
        minVal = np.min(dataSet)
        array = np.zeros(len(dataSet))
        for i in range(len(dataSet)):
            array[i] = (dataSet[i]-minVal)/(maxVal-minVal)
        return array
    def calculatePrecision(self,matrix):# calculate precision of predict
        sm = 0.0
        sumMatrix = np.sum(matrix,axis=0)
        for a in range(len(matrix)):
            
            tp = matrix[a][a]
            if sumMatrix[a]!=0 or tp!=0:
                sm += (tp/sumMatrix[a])
            
        return float("{:.6f}".format(sm/(len(matrix))))
    def calculateRecall(self,matrix):#calculate recall of predict
        sm = 0.0
        sumMatrix = np.sum(matrix,axis=1)
        for a in range(len(matrix)):
            tp = matrix[a][a]
            if sumMatrix[a]!=0 or tp!=0:
                sm += (tp/sumMatrix[a])
            
            
        return float("{:.6f}".format(sm/(len(matrix))))
        
    def calculateAccuracy(self,matrix): #calculate accuracy of predict
        diagsm = np.trace(matrix)
        allSum = np.sum(matrix)
            
    
        return float("{:.6f}".format(diagsm/allSum))
    
   
    def shuffle(self):#shuffle dataset
        rangex=10000
        ls = random.sample(range(rangex),rangex)
        shf = np.zeros((rangex,62))
        cnt = 0
        
        for a in ls:
            shf[cnt] = self.data[a]
            cnt += 1
        return shf
    

    def predict(self):
        shuffled = self.data
        if(self.featureScale == True):#check whether to do feature normalization
            for a in range(1,61):
                shuffled[:,a] = self.featureScaling(shuffled[:,a])
        datasize = len(shuffled)//self.fold
        splitted_x = shuffled[:,0:]
        
        accArr = self.accuracyTab
        preArr = self.precisionTab
        recArr = self.recallTab
        countPredictMistake = 1
        for k in range(self.fold):
            #split dataset
            x_train = np.concatenate((splitted_x[:(k * datasize)], splitted_x[((k + 1) * datasize):]))  
            x_valid = splitted_x[(k * datasize):][:datasize]  
            matrix = np.zeros((16,16))
            for d in x_valid:
                nw = np.zeros((self.kNN,63),dtype=float)
                count = 0
                for c in x_train:
                    length = self.euclidian_distance(c[1:-1], d[1:-1])
                    newArray = np.array([0.0])
                    newArray[0] = length
                    j = np.append(c, newArray)#add distance to the list
                    
                    if count < self.kNN:
                        nw[count] = j
                        if(count == self.kNN-1):
                            nw = nw[nw[:, -1].argsort()]
                    else:
                        if nw[-1][-1] > length:
                            nw[-1] = j
                            nw = nw[nw[:, -1].argsort()]

                    count += 1
                nw = nw[nw[:, -1].argsort()]
                
                skill = np.zeros(16,dtype=float)
                
                if (not self.weighted):#check whether to do weight
                    for z in nw:
                        skill[int(z[-2])] += 1
                        predicted = np.where(skill == np.max(skill))
                else:
                    for z in nw:
                        skill[int(z[-2])] += (1 / (z[-1]))
                        predicted = np.where(skill == np.max(skill))
                

                
                matrix[int(d[-1])][predicted[0][0]] += 1
            isWeighted=0
            isScale = 0
            if self.weighted :
                isWeighted = 0
            else:
                 isWeighted = 2
            if self.featureScale:
                isScale=0
            else:
                isScale=1
            strfold = "Fold "+str(k+1)
            
            #update tables
            accArr[k][((self.kNN*2)-2)+isWeighted+isScale]=self.calculateAccuracy(matrix)
            recArr[k][((self.kNN*2)-2)+isWeighted+isScale]=self.calculateRecall(matrix)
            preArr[k][((self.kNN*2)-2)+isWeighted+isScale]=self.calculatePrecision(matrix)
        
        
    def euclidian_distance(self,a,b):#calculate euclidian distance between 2 points
        result = np.float64(0)
        for i in range(len(a)):
            result = result + np.power((a[i] - b[i]), 2, dtype=np.float64)
        return sqrt(result)

In [4]:
class Regression():
    def __init__(self, kNN, fold, data,maeTab1,maeTab2, weighted=False,featureScale = False):
        self.kNN = kNN
        self.fold = fold
        self.weighted = weighted
        self.data = data
        self.maeTab1 = maeTab1
        self.maeTab2 = maeTab2
        self.weighted = weighted
        self.featureScale = featureScale
    
    def featureScaling(self,dataSet): 
        maxVal = np.max(dataSet)
        minVal = np.min(dataSet)
        array = np.zeros(len(dataSet))
        for i in range(len(dataSet)):
            array[i] = (dataSet[i]-minVal)/(maxVal-minVal)
        return array
    
    def shuffle(self):
        rangex=768 
        ls = random.sample(range(rangex),rangex)
        shf = np.zeros((rangex,10))
        cnt = 0

        for a in ls:
            shf[cnt] = self.data[a]
            cnt += 1
        return shf

    def euclidian_distance(self,a,b):
        result = np.float64(0)
        for i in range(len(a)):
            result = result + np.power((a[i] - b[i]), 2, dtype=np.float64)
        
        return sqrt(result)
    
    def meanAbsoluteError(self,predicted,gt):# calculate MAE
        sm = 0
        for a in range(len(predicted)):
            sm += abs(gt[a] - predicted[a])
            
        return float("{:.6f}".format((1/len(predicted)) * sm))
    
    def weightedFunc(self,matrix): 
        for a in range(len(matrix)):
            if not (a+1 >= len(matrix)):
                cross = matrix[a][0] * matrix[a+1][1] + matrix[a][1] * matrix[a+1][0]
                p = matrix[a][1] + matrix[a+1][1]
                matrix[a+1][0] = cross/p
                matrix[a+1][1] = p
        
        return matrix[len(matrix)-1][0]
    
    def predict(self):
        
        shuffled = self.data
        if(self.featureScale == True):
            for a in range(0,10):
                shuffled[:,a] = self.featureScaling(shuffled[:,a])
        datasize = len(shuffled)//self.fold
        splitted_x = shuffled[:,0:]
        mae1Arr = self.maeTab1
        mae2Arr = self.maeTab2
        for k in range(self.fold):
            #split dataset
            x_train = np.concatenate((splitted_x[:(k * datasize)], splitted_x[((k + 1) * datasize):]))  #train set
            x_valid = splitted_x[(k * datasize):][:datasize]  # test set
            cooling = np.zeros(len(x_valid))
            heating = np.zeros(len(x_valid))
            ct = 0
            for d in x_valid:
                nw = np.zeros((self.kNN,11),dtype=float)
                count = 0
                for c in x_train:
                    length = self.euclidian_distance(c[0:-2], d[0:-2])
                    newArray = np.array([0.0])
                    newArray[0] = length
                    j = np.append(c, newArray)
                    
                    if count < self.kNN:
                        nw[count] = j
                        if(count == self.kNN-1):
                            nw = nw[nw[:, -1].argsort()]
                            
                    else:
                        if float(nw[-1][-1]) > float(length):
                            nw[-1] = j
                            nw = nw[nw[:, -1].argsort()]

                    count += 1
                    
                nw = nw[nw[:, -1].argsort()]
                
                if(self.weighted == False):
                    heatingLoad = np.mean(nw,axis=0)[-3]
                    coolingLoad = np.mean(nw,axis=0)[-2]
                    cooling[ct] = coolingLoad
                    heating[ct] = heatingLoad
                    ct += 1
                else:
                    cooling[ct] =self.weightedFunc(nw[:,-2:])#cooling
                    heating[ct] =self.weightedFunc(nw[:,-3::2])#
                    ct += 1
                
                    
            
            meanAbsoluteErrorCooling = self.meanAbsoluteError(cooling,x_valid[:,-1])
            meanAbsoluteErrorHeating = self.meanAbsoluteError(heating,x_valid[:,-2])
                            
            
            isWeighted=0
            isScale = 0
            if self.weighted :
                isWeighted = 0
            else:
                 isWeighted = 2
            if self.featureScale:
                isScale=0
            else:
                isScale=1
            strfold = "Fold "+str(k+1)
            
            #update tables
            mae1Arr[k][((self.kNN*2)-2)+isWeighted+isScale]=meanAbsoluteErrorHeating
            
            mae2Arr[k][((self.kNN*2)-2)+isWeighted+isScale]=meanAbsoluteErrorCooling

                    
                    
                    

In [5]:
#PART I


df = np.array(dataFrame[0:]).astype(np.float64)

rangex=10000
ls = random.sample(range(rangex),rangex)
dfx = np.zeros((rangex,62))
cnt = 0

for a in ls:
    dfx[cnt] = df[a]
    cnt += 1

accuracyTab=np.zeros((5,20))
precisionTab=np.zeros((5,20))
recallTab=np.zeros((5,20))
kNNList = [1,3,5,7,9]
for a in kNNList:
    clf1 = Classification(a,5,df,accuracyTab,precisionTab,recallTab,True,True)
    clf1.predict()
    clf2 = Classification(a,5,df,accuracyTab,precisionTab,recallTab,True,False)
    clf2.predict()
    clf3 = Classification(a,5,df,accuracyTab,precisionTab,recallTab,False,True)
    clf3.predict()
    clf4 = Classification(a,5,df,accuracyTab,precisionTab,recallTab,False,False)
    clf4.predict()
average1=np.zeros((1,20))
for i in range(len(np.mean(accuracyTab,axis=0))):
    
    average1[0][i]=float("{:.6f}".format(np.mean(accuracyTab,axis=0)[i]))
    
average2=np.zeros((1,20))
for i in range(len(np.mean(precisionTab,axis=0))):
    average2[0][i]=float("{:.6f}".format(np.mean(precisionTab,axis=0)[i]))
    
average3=np.zeros((1,20))
for i in range(len(np.mean(recallTab,axis=0))):
    average3[0][i]=float("{:.6f}".format(np.mean(recallTab,axis=0)[i]))
    
#make dataframe table
accuracyTab=np.vstack([accuracyTab, average1])
precisionTab=np.vstack([precisionTab, average2])
recallTab=np.vstack([recallTab, average3])
tableAccuracy = pd.DataFrame(accuracyTab,columns=pd.MultiIndex.from_product([[1,3,5,7,9],['Yes', 'No'],["Yes","No"]],names = ["Nearest Neighbor","Weighted KNN","Feature Normalization"]),index = ["Fold 1","Fold 2","Fold 3","Fold 4","Fold 5","Average"])
tablePrecision = pd.DataFrame(precisionTab,columns=pd.MultiIndex.from_product([[1,3,5,7,9],['Yes', 'No'],["Yes","No"]],names = ["Nearest Neighbor","Weighted KNN","Feature Normalization"]),index = ["Fold 1","Fold 2","Fold 3","Fold 4","Fold 5","Average"])
tableRecall = pd.DataFrame(recallTab,columns=pd.MultiIndex.from_product([[1,3,5,7,9],['Yes', 'No'],["Yes","No"]],names = ["Nearest Neighbor","Weighted KNN","Feature Normalization"]),index = ["Fold 1","Fold 2","Fold 3","Fold 4","Fold 5","Average"])
tableAccuracy.to_csv("accSave.csv")
tablePrecision.to_csv("preSave.csv")
tableRecall.to_csv("recSave.csv")

In [6]:
#PART II


maeTab1=np.zeros((5,20))#heating
maeTab2=np.zeros((5,20))


df3 = np.array(dataFrame2[:])
rangex=768
ls = random.sample(range(rangex),rangex)
df2 = np.zeros((rangex,10))
cnt = 0

for a in ls:
    df2[cnt] = df3[a]
    cnt += 1

kNNList = [1,3,5,7,9]
for a in kNNList:
    clf1 = Regression(a,5,df2,maeTab1,maeTab2,True,True)
    clf1.predict()
    clf2 = Regression(a,5,df2,maeTab1,maeTab2,True,False)
    clf2.predict()
    clf3 = Regression(a,5,df2,maeTab1,maeTab2,False,True)
    clf3.predict()
    clf4 = Regression(a,5,df2,maeTab1,maeTab2,False,False)
    clf4.predict()
average1=np.zeros((1,20))
for i in range(len(np.mean(maeTab1,axis=0))):
    average1[0][i]=float("{:.6f}".format(np.mean(maeTab1,axis=0)[i]))


average2=np.zeros((1,20))
for i in range(len(np.mean(maeTab2,axis=0))):
    average2[0][i]=float("{:.6f}".format(np.mean(maeTab2,axis=0)[i]))
#make dataframe table
maeTab1=np.vstack([maeTab1, average1 ])
maeTab2=np.vstack([maeTab2, average2])

tableHeating = pd.DataFrame(maeTab1,columns=pd.MultiIndex.from_product([[1,3,5,7,9],['Yes', 'No'],["Yes","No"]],names = ["Nearest Neighbor","Weighted KNN","Feature Normalization"]),index = ["Fold 1","Fold 2","Fold 3","Fold 4","Fold 5","Average"])
tableCooling = pd.DataFrame(maeTab2,columns=pd.MultiIndex.from_product([[1,3,5,7,9],['Yes', 'No'],["Yes","No"]],names = ["Nearest Neighbor","Weighted KNN","Feature Normalization"]),index = ["Fold 1","Fold 2","Fold 3","Fold 4","Fold 5","Average"])

tableHeating.to_csv("heating.csv")
tableCooling.to_csv("cooling.csv")



In [7]:
pd.set_option('display.max_columns', None)
accuracydf=pd.read_csv("accSave.csv",index_col=0,header=[0,1,2])
accuracydf.columns=pd.MultiIndex.from_product([[1,3,5,7,9],['Yes', 'No'],["Yes","No"]],names = ["Nearest Neighbor","Weighted KNN","Feature Normalization"])
precisiondf=pd.read_csv("preSave.csv",index_col=0,header=[0,1,2])
precisiondf.columns=pd.MultiIndex.from_product([[1,3,5,7,9],['Yes', 'No'],["Yes","No"]],names = ["Nearest Neighbor","Weighted KNN","Feature Normalization"])
recalldf=pd.read_csv("recSave.csv",index_col=0,header=[0,1,2])
recalldf.columns=pd.MultiIndex.from_product([[1,3,5,7,9],['Yes', 'No'],["Yes","No"]],names = ["Nearest Neighbor","Weighted KNN","Feature Normalization"])

tableHeatingdf=pd.read_csv("heating.csv",index_col=0,header=[0,1,2])
tableHeatingdf.columns=pd.MultiIndex.from_product([[1,3,5,7,9],['Yes', 'No'],["Yes","No"]],names = ["Nearest Neighbor","Weighted KNN","Feature Normalization"])
tableCoolingdf=pd.read_csv("cooling.csv",index_col=0,header=[0,1,2])
tableCoolingdf.columns=pd.MultiIndex.from_product([[1,3,5,7,9],['Yes', 'No'],["Yes","No"]],names = ["Nearest Neighbor","Weighted KNN","Feature Normalization"])

# CONCLUSION
The difference between each fold depends on shuffling. If shuffle once more, it might change. 
The averages when k = 1 are the same. Because the closest element will not be changed.

## Accuracy


In [8]:
display(accuracydf)

Nearest Neighbor            1                               3                  \
Weighted KNN              Yes              No             Yes              No   
Feature Normalization     Yes      No     Yes      No     Yes      No     Yes   
Fold 1                 0.9605  0.9605  0.9605  0.9605  0.9790  0.9790  0.9770   
Fold 2                 0.9680  0.9680  0.9680  0.9680  0.9825  0.9825  0.9805   
Fold 3                 0.9660  0.9660  0.9660  0.9660  0.9795  0.9795  0.9805   
Fold 4                 0.9660  0.9660  0.9660  0.9660  0.9825  0.9825  0.9810   
Fold 5                 0.9570  0.9570  0.9570  0.9570  0.9795  0.9795  0.9775   
Average                0.9635  0.9635  0.9635  0.9635  0.9806  0.9806  0.9793   

Nearest Neighbor                    5                               7          \
Weighted KNN                      Yes              No             Yes           
Feature Normalization      No     Yes      No     Yes      No     Yes      No   
Fold 1                 0.9770  0.9825  0.9825  0.9825  0.9825  0.9835  0.9835   
Fold 2                 0.9805  0.9865  0.9865  0.9855  0.9855  0.9875  0.9875   
Fold 3                 0.9805  0.9820  0.9820  0.9800  0.9800  0.9850  0.9850   
Fold 4                 0.9810  0.9865  0.9865  0.9870  0.9870  0.9880  0.9880   
Fold 5                 0.9775  0.9825  0.9825  0.9825  0.9825  0.9835  0.9835   
Average                0.9793  0.9840  0.9840  0.9835  0.9835  0.9855  0.9855   

Nearest Neighbor                            9                          
Weighted KNN               No             Yes              No          
Feature Normalization     Yes      No     Yes      No     Yes      No  
Fold 1                 0.9835  0.9835  0.9845  0.9845  0.9845  0.9845  
Fold 2                 0.9885  0.9885  0.9875  0.9875  0.9875  0.9875  
Fold 3                 0.9855  0.9855  0.9860  0.9860  0.9855  0.9855  
Fold 4                 0.9880  0.9880  0.9885  0.9885  0.9870  0.9870  
Fold 5                 0.9835  0.9835  0.9840  0.9840  0.9820  0.9820  
Average                0.9858  0.9858  0.9861  0.9861  0.9853  0.9853


**Does feature normalization effect the accuracy?**
As we can see, doesn't matter if it is weighted and what k value is. Feature normalization doesn't effect the predicts.
Because of that reason, accuracy doesn't change as well.

**How does weighted/unweighted situation change the accuracy?**
When we check the table, they look like equal but they are not. There are differences between them. It doesn't have 
to effect everytime ,but it effects. It depends on the data we look at. For example: Fold 2 , k = 3 it is changed. 

**How does k value effect the accuracy?**
We can easily see that accuracy increases usually if we get bigger k. It might change because of the data we get.
For example: Fold 1, k=7 Yes,Yes = 0,9855 and k=9, Yes,Yes = 0,9845.

## Precision/Recall

In [9]:
display(precisiondf)

Nearest Neighbor              1                                       3  \
Weighted KNN                Yes                  No                 Yes   
Feature Normalization       Yes        No       Yes        No       Yes   
Fold 1                 0.960366  0.960366  0.960366  0.960366  0.978910   
Fold 2                 0.968027  0.968027  0.968027  0.968027  0.982444   
Fold 3                 0.966368  0.966368  0.966368  0.966368  0.979601   
Fold 4                 0.965383  0.965383  0.965383  0.965383  0.982213   
Fold 5                 0.958551  0.958551  0.958551  0.958551  0.979803   
Average                0.963739  0.963739  0.963739  0.963739  0.980594   

Nearest Neighbor                                            5            \
Weighted KNN                           No                 Yes             
Feature Normalization        No       Yes        No       Yes        No   
Fold 1                 0.978910  0.977017  0.977017  0.982496  0.982496   
Fold 2                 0.982444  0.980453  0.980453  0.986673  0.986673   
Fold 3                 0.979601  0.980691  0.980691  0.982383  0.982383   
Fold 4                 0.982213  0.980506  0.980506  0.986269  0.986269   
Fold 5                 0.979803  0.977823  0.977823  0.982812  0.982812   
Average                0.980594  0.979298  0.979298  0.984127  0.984127   

Nearest Neighbor                                  7                      \
Weighted KNN                 No                 Yes                  No   
Feature Normalization       Yes        No       Yes        No       Yes   
Fold 1                 0.982483  0.982483  0.983555  0.983555  0.983436   
Fold 2                 0.985510  0.985510  0.987695  0.987695  0.988813   
Fold 3                 0.980646  0.980646  0.985261  0.985261  0.985926   
Fold 4                 0.986689  0.986689  0.987823  0.987823  0.987846   
Fold 5                 0.982694  0.982694  0.983748  0.983748  0.983748   
Average                0.983604  0.983604  0.985616  0.985616  0.985954   

Nearest Neighbor                        9                                
Weighted KNN                          Yes                  No            
Feature Normalization        No       Yes        No       Yes        No  
Fold 1                 0.983436  0.984393  0.984393  0.984253  0.984253  
Fold 2                 0.988813  0.987767  0.987767  0.987898  0.987898  
Fold 3                 0.985926  0.986279  0.986279  0.985767  0.985767  
Fold 4                 0.987846  0.988282  0.988282  0.986693  0.986693  
Fold 5                 0.983748  0.984192  0.984192  0.982316  0.982316  
Average                0.985954  0.986183  0.986183  0.985385  0.985385

In [10]:
display(recalldf)

Nearest Neighbor              1                                       3  \
Weighted KNN                Yes                  No                 Yes   
Feature Normalization       Yes        No       Yes        No       Yes   
Fold 1                 0.959885  0.959885  0.959885  0.959885  0.978814   
Fold 2                 0.968997  0.968997  0.968997  0.968997  0.983165   
Fold 3                 0.966067  0.966067  0.966067  0.966067  0.979654   
Fold 4                 0.966591  0.966591  0.966591  0.966591  0.982539   
Fold 5                 0.956394  0.956394  0.956394  0.956394  0.979172   
Average                0.963587  0.963587  0.963587  0.963587  0.980669   

Nearest Neighbor                                            5            \
Weighted KNN                           No                 Yes             
Feature Normalization        No       Yes        No       Yes        No   
Fold 1                 0.978814  0.976669  0.976669  0.982360  0.982360   
Fold 2                 0.983165  0.981116  0.981116  0.986865  0.986865   
Fold 3                 0.979654  0.980708  0.980708  0.982115  0.982115   
Fold 4                 0.982539  0.981004  0.981004  0.986491  0.986491   
Fold 5                 0.979172  0.977282  0.977282  0.982156  0.982156   
Average                0.980669  0.979356  0.979356  0.983997  0.983997   

Nearest Neighbor                                  7                      \
Weighted KNN                 No                 Yes                  No   
Feature Normalization       Yes        No       Yes        No       Yes   
Fold 1                 0.982435  0.982435  0.983322  0.983322  0.983329   
Fold 2                 0.985784  0.985784  0.987670  0.987670  0.988622   
Fold 3                 0.979936  0.979936  0.985193  0.985193  0.985572   
Fold 4                 0.986947  0.986947  0.987958  0.987958  0.988019   
Fold 5                 0.982250  0.982250  0.983180  0.983180  0.983180   
Average                0.983470  0.983470  0.985465  0.985465  0.985744   

Nearest Neighbor                        9                                
Weighted KNN                          Yes                  No            
Feature Normalization        No       Yes        No       Yes        No  
Fold 1                 0.983329  0.984284  0.984284  0.984339  0.984339  
Fold 2                 0.988622  0.987653  0.987653  0.987727  0.987727  
Fold 3                 0.985572  0.986079  0.986079  0.985544  0.985544  
Fold 4                 0.988019  0.988530  0.988530  0.987108  0.987108  
Fold 5                 0.983180  0.983759  0.983759  0.981820  0.981820  
Average                0.985744  0.986061  0.986061  0.985308  0.985308

**Does feature normalization effect the precision?**
Feature normalization doesn't effect anything here too, because the closest k elements are the same.

**How does weighted/unweighted situation ?**
It effects positively.It effects a little for precision too.

**How does k value effect the precision ?**
It usually increases when we get bigger k but there are some coincidences here too.

## Mean Absolute Error 


In [11]:
display(tableHeatingdf)

Nearest Neighbor              1                                       3  \
Weighted KNN                Yes                  No                 Yes   
Feature Normalization       Yes        No       Yes        No       Yes   
Fold 1                 0.073103  2.411634  0.094992  2.903137  0.055389   
Fold 2                 0.092707  2.584314  0.099347  2.462614  0.057538   
Fold 3                 0.086386  2.630000  0.093405  2.359020  0.057422   
Fold 4                 0.091461  2.842941  0.086811  2.477582  0.053140   
Fold 5                 0.088201  2.435359  0.073346  2.600980  0.052830   
Average                0.086372  2.580850  0.089580  2.560667  0.055264   

Nearest Neighbor                                            5            \
Weighted KNN                           No                 Yes             
Feature Normalization        No       Yes        No       Yes        No   
Fold 1                 1.862157  0.053595  1.822505  0.050674  1.890489   
Fold 2                 1.809606  0.048548  1.627190  0.064221  1.564533   
Fold 3                 2.150161  0.054265  1.707059  0.066820  2.012996   
Fold 4                 2.086099  0.051027  1.802266  0.072267  1.975983   
Fold 5                 1.908261  0.050673  1.811176  0.063527  1.686223   
Average                1.963257  0.051622  1.754039  0.063502  1.826045   

Nearest Neighbor                                  7                      \
Weighted KNN                 No                 Yes                  No   
Feature Normalization       Yes        No       Yes        No       Yes   
Fold 1                 0.049173  1.516745  0.062318  2.871524  0.047225   
Fold 2                 0.058749  1.517229  0.063818  2.990954  0.054081   
Fold 3                 0.048747  1.719908  0.054244  2.847635  0.054269   
Fold 4                 0.058929  1.591556  0.053672  2.793953  0.045153   
Fold 5                 0.051489  1.527438  0.056475  2.789158  0.047058   
Average                0.053417  1.574575  0.058105  2.858645  0.049557   

Nearest Neighbor                        9                                
Weighted KNN                          Yes                  No            
Feature Normalization        No       Yes        No       Yes        No  
Fold 1                 1.773810  0.062172  3.279616  0.054777  1.964401  
Fold 2                 1.879066  0.068681  3.180834  0.048659  2.086696  
Fold 3                 2.051018  0.059024  3.224047  0.048337  1.920102  
Fold 4                 1.803725  0.062945  3.590579  0.048164  2.510675  
Fold 5                 1.738702  0.065395  3.800469  0.052993  1.904960  
Average                1.849264  0.063643  3.415109  0.050586  2.077367

In [12]:
display(tableCoolingdf)

Nearest Neighbor              1                                       3  \
Weighted KNN                Yes                  No                 Yes   
Feature Normalization       Yes        No       Yes        No       Yes   
Fold 1                 0.082376  1.872614  0.105202  2.383007  0.054400   
Fold 2                 0.095453  2.145425  0.103415  2.100523  0.059267   
Fold 3                 0.094774  2.200327  0.098434  1.974837  0.055373   
Fold 4                 0.099910  2.318693  0.091132  2.173791  0.059581   
Fold 5                 0.096862  2.050196  0.083374  2.096863  0.057302   
Average                0.093875  2.117451  0.096311  2.145804  0.057185   

Nearest Neighbor                                            5            \
Weighted KNN                           No                 Yes             
Feature Normalization        No       Yes        No       Yes        No   
Fold 1                 1.426798  0.058188  1.598497  0.061280  1.980021   
Fold 2                 1.519033  0.059735  1.484444  0.078392  1.628170   
Fold 3                 1.730705  0.057315  1.617582  0.075176  1.995019   
Fold 4                 1.624781  0.057842  1.417843  0.076839  1.950945   
Fold 5                 1.441175  0.056224  1.633290  0.070431  1.884686   
Average                1.548498  0.057861  1.550331  0.072424  1.887768   

Nearest Neighbor                                  7                      \
Weighted KNN                 No                 Yes                  No   
Feature Normalization       Yes        No       Yes        No       Yes   
Fold 1                 0.050026  1.300366  0.071505  2.353872  0.048143   
Fold 2                 0.065988  1.534366  0.070448  2.778615  0.060037   
Fold 3                 0.054874  1.609425  0.058253  2.844235  0.061783   
Fold 4                 0.065575  1.330667  0.067025  2.608464  0.053699   
Fold 5                 0.057821  1.451229  0.062422  2.453848  0.051565   
Average                0.058857  1.445211  0.065931  2.607807  0.055045   

Nearest Neighbor                        9                                
Weighted KNN                          Yes                  No            
Feature Normalization        No       Yes        No       Yes        No  
Fold 1                 1.636807  0.068881  3.030718  0.059146  1.685156  
Fold 2                 1.706013  0.078245  2.688443  0.056991  1.948155  
Fold 3                 1.754454  0.076126  2.947927  0.050837  1.696144  
Fold 4                 1.622194  0.071705  3.124990  0.054432  1.969702  
Fold 5                 1.449038  0.070454  3.334669  0.058521  1.877560  
Average                1.633701  0.073082  3.025349  0.055985  1.835343

**Does feature normalization effect the MAE?**
It effects the MAE too much unlike the accuracy,precision and recall for the first part.The MAE without feature normalization   is almost 50 times bigger than The MAE with feature normalization.It's reason is there is a big difference between the max and
min value of the dataset.

**How does weighted/unweighted situation effect the MAE?**
It has no big affect like feature normalization, but it has affect too. The reason of changing is that there is inverse proportional between weight and length

**How does k value effect the MAE?**
It doesn't effect regularly. It's affect changes usually.

## Our Comment

The results are better than we expected. Accuracy average is 0.97, it is too high when we consider that artificial intelligence predict results by using test data. In our opinion the reason of high result is the artificiality of given dataset. 

The hardest part of this assignment for us is part2,weighted part. Generating distance calculator method was hard for us. 